In [87]:
# Packages

import json
import pandas as pd
import tweepy
import time

In [88]:
def to_epoch(tweet_time_string):
    from email.utils import parsedate_tz, mktime_tz
    """Convert rfc 5322 -like time string into epoch

    """
    return mktime_tz(parsedate_tz(tweet_time_string))

In [89]:
# Credentials

consumer_key = '' 
consumer_secret = ''
access_key = ''
access_secret = ''

In [115]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)
api = tweepy.API(auth, wait_on_rate_limit_notify=True, wait_on_rate_limit=True)

In [145]:
query1 = '"chicago booth" -filter:retweets'
query2 = '"northwestern kellogg"'
query3 = '"elon musk" -filter:retweets'
query4 = '"university of chicago" -filter:retweets'

In [92]:
boothTweets = tweepy.Cursor(api.search, q = query1, lang = 'en').items()
kelloggTweets = tweepy.Cursor(api.search, q = query2, lang = 'en').items()

In [ ]:
def get_tweets(query):
    output = []
    response = tweepy.Cursor(api.search, q = query, lang = 'en', count = 100)
    for tweet in response.items():
        temp = tweet._json
        temp['keywords'] = query.strip('"')
        temp['created_at_epoch'] = to_epoch(temp['created_at'])
        temp['scraped_at_epoch'] = int(time.time())
        temp['t'] = int((temp['scraped_at_epoch'] - temp['created_at_epoch'])/1800)
        output.append(temp)
    return output

In [36]:
def get_DF(list_of_tweets):
    keys = ['id_str', 'created_at_epoch', 'scraped_at_epoch', 'retweet_count', 'favorite_count', 'keywords']
    tempdict = {}
    for key in keys:
        templist = []
        for tweet in list_of_tweets:
            templist.append(tweet.get(key))
        tempdict[key] = templist
    return tempdict

In [33]:
import pandas as pd

In [46]:
boothDF = pd.DataFrame.from_dict(get_DF(booth_tweets))

In [47]:
kelloggDF = pd.DataFrame.from_dict(get_DF(kellogg_tweets))

In [72]:
boothDF.to_csv('booth.csv')

In [73]:
kelloggDF.to_csv('kellogg.csv')

In [42]:
booth_tweets = get_tweets(query1)
kellogg_tweets = get_tweets(query2)

In [45]:
boothDF = pd.DataFrame.from_dict(get_DF(booth_tweets))
kelloggDF = pd.DataFrame.from_dict(get_DF(kellogg_tweets))

In [50]:
with open('booth.csv', 'a') as f:
    boothDF.to_csv(f, header=False)

In [51]:
with open('kellogg.csv', 'a') as f:
    kelloggDF.to_csv(f, header=False)

In [55]:
def pipeline():
    try:
    booth_tweets = get_tweets(query1)
    kellogg_tweets = get_tweets(query2)
    boothDF = pd.DataFrame.from_dict(get_DF(booth_tweets))
    kelloggDF = pd.DataFrame.from_dict(get_DF(kellogg_tweets))
    with open('booth.csv', 'a') as f:
        boothDF.to_csv(f, header=False)
    with open('kellogg.csv', 'a') as f:
        kelloggDF.to_csv(f, header=False)

In [68]:
import schedule

In [70]:
schedule.every(30).minutes.do(pipeline)

Every 30 minutes do pipeline() (last run: [never], next run: 2018-12-03 00:54:41)

In [74]:
while True:
    schedule.run_pending()
    time.sleep(1)

TweepError: Failed to send request: HTTPSConnectionPool(host='api.twitter.com', port=443): Max retries exceeded with url: /1.1/search/tweets.json?q=%22chicago+booth%22&lang=en (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x11d5af4a8>: Failed to establish a new connection: [Errno 50] Network is down',))